 Tout d'abord,importons les librairies et modules dont nous avons besoin.

In [3]:
import numpy
from matplotlib import pyplot
%matplotlib inline

Implémentons maintenant la méthode "Runge-Kutta 4". Notons ici que les positions et les vitesses sont stockées dans deux matrices 2 x n, de même pour les dx et dv.

In [ ]:
dx1 = h*v[0,:] #Les lignes paires sont les vitesses selon x
dy1 = h*v[1,:] #Les lignes impaires sont les vitesses selon y

